# 1. 환경설정

In [1]:
import pickle
import time
import pandas as pd
import pickle
from pykiwoom.kiwoom import *
import warnings
warnings.filterwarnings('ignore')
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

In [2]:
state = kiwoom.GetConnectState()
if state == 0:
    print("미연결")
elif state == 1:
    print("연결완료")

연결완료


# 2. 데이터 읽기

In [5]:
with open('sector.pickle','rb') as f:
    sector = pickle.load(f)
with open('df_sector2.pickle','rb') as f:
    df_sector = pickle.load(f)
#with open('kospi.pickle','rb') as f:
 #   kospi = pickle.load(f)

In [6]:
sector

{'Food': ['097950', '271560', '000080', '004370', '005300'],
 'Clothing': ['383220', '093050', '020000', '105630', '001070'],
 'Chemical': ['051910', '096770', '010950', '051900', '090430'],
 'Medicine': ['207940', '068270', '302440', '000100', '128940'],
 'Non_Metal': ['003670', '003410', '010780', '300720'],
 'Metal': ['005490', '010130', '004020', '016380', '001230'],
 'Machine': ['034020', '018880', '241560', '112610'],
 'Electronic': ['005930', '000660', '373220', '006400', '066570'],
 'Construction': ['000720', '006360', '047040', '051600'],
 'Transport': ['011200', '003490', '086280', '180640', '028670'],
 'Distribution': ['028260', '023530', '282330', '139480', '004170'],
 'Power': ['015760', '036460', '017390'],
 'Tele': ['017670', '030200', '032640'],
 'Finance': ['323410', '003550', '000810', '377300', '006800'],
 'Brokerage': ['005940', '016360', '008560', '039490'],
 'Insurer': ['032830', '005830', '000060', '001450'],
 'Service': ['035420', '035720', '259960', '018260', '

In [7]:
df_sector

{'Food': {'097950':              일자    거래량      시가      고가      저가      종가
  0    2022-07-18  27816  396000  399000  389000  391000
  1    2022-07-15  40227  395000  401000  381000  397500
  2    2022-07-14  34230  399000  405500  395000  402000
  3    2022-07-13  50183  400000  407500  396500  403000
  4    2022-07-12  48509  389500  398500  388500  398500
  ...         ...    ...     ...     ...     ...     ...
  2995 2010-05-27  46323  208500  212000  205000  211500
  2996 2010-05-26  77164  212500  213500  203500  213500
  2997 2010-05-25  71590  210000  216500  199000  213000
  2998 2010-05-24  28817  211000  218500  210500  211000
  2999 2010-05-20  57088  214500  220000  213500  214000
  
  [3000 rows x 6 columns],
  '271560':              일자      거래량      시가      고가      저가      종가
  0    2022-07-18   140201  106500  107000  103500  104000
  1    2022-07-15    90893  108500  108500  104000  106500
  2    2022-07-14   138391  108000  109500  106000  107000
  3    2022-07-13   20

In [8]:
#kospi

# 3. 데이터셋 최신화
1. 최신 주가 데이터 읽고 업종 데이터셋 추가
2. 최신 코스피 지수 읽고 코스피 데이터셋 추가

In [9]:
from datetime import date,datetime
today = date.today()
today = today.isoformat()
today = today.replace('-','')
yesterday = int(today)-1
yesterday = str(yesterday)
print(yesterday, type(yesterday))
set_d = yesterday

20220718 <class 'str'>


In [10]:
today

'20220719'

In [11]:
set_d

'20220718'

# 최신 data 수집

In [12]:
df_sector_2 = {'Food':{},
          'Clothing' : {},
          'Chemical' : {},
          'Medicine' : {},
          'Non_Metal' : {},
          'Metal': {},
          'Machine' : {},
          'Electronic' : {},
          'Construction' : {},
          'Transport': {},
          'Distribution' : {},
          'Power': {},
          'Tele' : {},
          'Finance': {},
          'Brokerage' : {},
          'Insurer': {},
          'Service': {},
          'Manufacturer': {}}

In [13]:
def Add_Data(sector_name,stockcode):
    df = kiwoom.block_request("opt10081",
                          종목코드=stockcode,
                          기준일자=set_d,
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)
    df = df[['일자','현재가','거래량','시가','고가','저가']]
    df2 = pd.DataFrame()
    df2['일자'] = pd.to_datetime(df['일자'])
    df2['종가'] = df['현재가'].astype(int)
    df2['거래량'] = df['거래량'].astype(int)
    df2['시가'] = df['시가'].astype(int)
    df2['고가'] = df['고가'].astype(int)
    df2['저가'] = df['저가'].astype(int)
    df_sector_2[sector_name][stockcode] = df2
    del df2

In [14]:
import time

In [15]:
#데이터 수집 자동화 코드
for sector_name, stockcode_list in sector.items():
    print(f'---------{sector_name}업종 데이터 수집----------')
    for stockcode in stockcode_list:
        # 주식항목이름
        stockname = kiwoom.GetMasterCodeName(stockcode)
        print(f'--------[{stockname}] 항목 데이터 수집---------')
        Add_Data(sector_name,stockcode)
        time.sleep(0.2) #time을 안넣어주면 중간에 종종 수집이 안되고 멈추는 경우가 발생

---------Food업종 데이터 수집----------
--------[CJ제일제당] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[오리온] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[하이트진로] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101


TypeError: 'NoneType' object is not subscriptable

--------[농심] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[롯데칠성] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
---------Clothing업종 데이터 수집----------
--------[F&F] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[LF] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[한섬] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[한세실업] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101


TypeError: 'NoneType' object is not subscriptable

--------[대한방직] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
---------Chemical업종 데이터 수집----------
--------[LG화학] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[SK이노베이션] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[S-Oil] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[LG생활건강] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[아모레퍼시픽] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101


...

---------Tele업종 데이터 수집----------
--------[SK텔레콤] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[KT] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[LG유플러스] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
---------Finance업종 데이터 수집----------
--------[카카오뱅크] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[LG] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[삼성화재] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[카카오페이] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[미래에셋증권] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101


TypeError: 'NoneType' object is not subscriptable

---------Brokerage업종 데이터 수집----------
--------[NH투자증권] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[삼성증권] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[메리츠증권] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[키움증권] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
---------Insurer업종 데이터 수집----------
--------[삼성생명] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[DB손해보험] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[메리츠화재] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[현대해상] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
---------Service업종 데이터 수집----------
--------[NAVER] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[카카오] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[크래프톤] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[삼성에스디에스] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[엔씨소프트] 항목 데이터 수집---------
CommRq

TypeError: 'NoneType' object is not subscriptable

--------[현대모비스] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[현대중공업] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[KT&G] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101
--------[삼성전기] 항목 데이터 수집---------
CommRqData opt10081 opt10081 0 0101


In [16]:
df_sector_2

{'Food': {'097950':             일자      종가    거래량      시가      고가      저가
  0   2022-07-18  391000  27816  396000  399000  389000
  1   2022-07-15  397500  40227  395000  401000  381000
  2   2022-07-14  402000  34230  399000  405500  395000
  3   2022-07-13  403000  50183  400000  407500  396500
  4   2022-07-12  398500  48509  389500  398500  388500
  ..         ...     ...    ...     ...     ...     ...
  595 2020-02-20  273500  37609  274000  274500  269000
  596 2020-02-19  274500  64463  272500  278000  272000
  597 2020-02-18  268500  43875  271500  271500  263500
  598 2020-02-17  271000  51552  274000  278500  269500
  599 2020-02-14  276000  64161  275500  279000  267000
  
  [600 rows x 6 columns],
  '271560':             일자      종가     거래량      시가      고가      저가
  0   2022-07-18  104000  140201  106500  107000  103500
  1   2022-07-15  106500   90893  108500  108500  104000
  2   2022-07-14  107000  138391  108000  109500  106000
  3   2022-07-13  109000  203292  108000  1

# 이전 data와 최신 data 병합(최신화)

In [15]:
'''
df_sector_3 = {'Food':{},
          'Clothing' : {},
          'Chemical' : {},
          'Medicine' : {},
          'Non_Metal' : {},
          'Metal': {},
          'Machine' : {},
          'Electronic' : {},
          'Construction' : {},
          'Transport': {},
          'Distribution' : {},
          'Power': {},
          'Tele' : {},
          'Finance': {},
          'Brokerage' : {},
          'Insurer': {},
          'Service': {},
          'Manufacturer': {}}
'''

In [17]:
for sector_name, df_dict in df_sector.items():
    for stockcode, df in df_dict.items():
        df = pd.concat([df_sector_2[sector_name][stockcode],df])
        df=df.drop_duplicates(subset=['일자'])
        df = df.reset_index(drop=True)
        df_sector[sector_name][stockcode] = df

In [18]:
df_sector

{'Food': {'097950':              일자      종가    거래량      시가      고가      저가
  0    2022-07-18  391000  27816  396000  399000  389000
  1    2022-07-15  397500  40227  395000  401000  381000
  2    2022-07-14  402000  34230  399000  405500  395000
  3    2022-07-13  403000  50183  400000  407500  396500
  4    2022-07-12  398500  48509  389500  398500  388500
  ...         ...     ...    ...     ...     ...     ...
  2995 2010-05-27  211500  46323  208500  212000  205000
  2996 2010-05-26  213500  77164  212500  213500  203500
  2997 2010-05-25  213000  71590  210000  216500  199000
  2998 2010-05-24  211000  28817  211000  218500  210500
  2999 2010-05-20  214000  57088  214500  220000  213500
  
  [3000 rows x 6 columns],
  '271560':              일자      종가      거래량      시가      고가      저가
  0    2022-07-18  104000   140201  106500  107000  103500
  1    2022-07-15  106500    90893  108500  108500  104000
  2    2022-07-14  107000   138391  108000  109500  106000
  3    2022-07-13  109

# 최신 kospi 수집

In [18]:
'''
def Add_Kospi():
    df = kiwoom.block_request("opt20006",
                              업종코드="001",
                              기준일자=set_d,
                              output="업종일봉조회",
                              next=0)
    df = df[['일자','현재가']]
    df2 = pd.DataFrame()
    df2['일자'] = pd.to_datetime(df['일자'])
    df2['코스피지수'] = df['현재가'].astype(int)
    df2['코스피지수'] = df2['코스피지수'].apply(lambda x : x/100)
    return df2    
    
'''

In [19]:
#kospi2 = Add_Kospi()

CommRqData opt20006 opt20006 0 0101


In [20]:
#kospi2

,일자,코스피지수
0,2022-07-15,2330.98
1,2022-07-14,2322.32
2,2022-07-13,2328.61
3,2022-07-12,2317.76
4,2022-07-11,2340.27
...,...,...
595,2020-02-19,2210.34
596,2020-02-18,2208.88
597,2020-02-17,2242.17
598,2020-02-14,2243.59


# 이전 kospi와 최신 kospi 병합(최신화)

In [21]:
#kospi3 = pd.concat([kospi2,kospi])
#kospi3=kospi3.drop_duplicates(subset=['일자'])
#kospi3 = kospi3.reset_index(drop=True)

In [22]:
#kospi3

,일자,코스피지수
0,2022-07-15,2330.98
1,2022-07-14,2322.32
2,2022-07-13,2328.61
3,2022-07-12,2317.76
4,2022-07-11,2340.27
...,...,...
3000,2010-05-18,1643.24
3001,2010-05-17,1651.51
3002,2010-05-14,1695.63
3003,2010-05-13,1694.58


In [23]:
with open('df_sector_UTD.pickle','wb') as f:
    pickle.dump(df_sector,f)

In [24]:
# with open('kospi.pickle','wb') as f:
#     pickle.dump(kospi3,f)